In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.neighbors import NearestNeighbors
# from ipynb.fs.full.Preprocessing import clean_data
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:
# Tracks file with ids, titles, names, etc.
tracks = pd.read_csv('Data/tracks.csv')
print(tracks.shape)
tracks.head()

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [7]:
# New cleaned songs file with all features as one-hot categorical variables 
df = pd.read_csv('Data/new_songs_cleaned.csv', index_col='Unnamed: 0')
print(df.shape)
df.head()

(586344, 74)


,popularity_1,popularity_2,popularity_3,popularity_4,popularity_5,duration_ms_1,duration_ms_2,duration_ms_3,duration_ms_4,duration_ms_5,...,valence_5,tempo_1,tempo_2,tempo_3,tempo_4,tempo_5,time_signature_1,time_signature_2,time_signature_3,time_signature_4
0,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [8]:
# Smaller dataframe for rapid testing

# small_new_df = new_df.iloc[:150000]
# print(small_new_df.shape)
# small_new_df.head()

In [10]:
# Autoencoder using new_songs_cleaned (all categorical) VAL_LOSS = 0.0015
data = df

encoded_dim = 16

input_song = Input(shape = (data.shape[1], ))
h1 = Dense(32, activation = 'relu')(input_song)
encoded = Dense(encoded_dim, activation = 'relu')(h1)

dh1 = Dense(32, activation='sigmoid')(encoded)
decoded = Dense(data.shape[1], activation='sigmoid')(dh1)

autoencoder = Model(input_song, decoded)
encoder = Model(input_song, encoded)

opt = tf.keras.optimizers.Nadam()
autoencoder.compile(optimizer=opt, loss='mean_squared_error')

stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, restore_best_weights=True)

autoencoder.fit(data, 
                data,  
                batch_size=32,
                epochs=100,
                validation_split=.2,
                callbacks=[stop])

Epoch 1/100
14659/14659 [==============================] - 12s 766us/step - loss: 0.0376 - val_loss: 0.0238
Epoch 2/100
14659/14659 [==============================] - 11s 735us/step - loss: 0.0197 - val_loss: 0.0175
Epoch 3/100
14659/14659 [==============================] - 11s 753us/step - loss: 0.0146 - val_loss: 0.0132
Epoch 4/100
14659/14659 [==============================] - 12s 846us/step - loss: 0.0111 - val_loss: 0.0104
Epoch 5/100
14659/14659 [==============================] - 11s 750us/step - loss: 0.0088 - val_loss: 0.0086
Epoch 6/100
14659/14659 [==============================] - 11s 741us/step - loss: 0.0075 - val_loss: 0.0074
Epoch 7/100
14659/14659 [==============================] - 13s 919us/step - loss: 0.0066 - val_loss: 0.0066
Epoch 8/100
14659/14659 [==============================] - 13s 921us/step - loss: 0.0061 - val_loss: 0.0061
Epoch 9/100
14659/14659 [==============================] - 12s 838us/step - loss: 0.0055 - val_loss: 0.0055
Epoch 10/100
14659/14659 [==

In [13]:
# Get song encodings
data = df
decoded_songs = autoencoder.predict(data)
encoded_songs = encoder.predict(data)

embeddings_df = pd.DataFrame(encoded_songs, index=data.index)
print(embeddings_df.shape)
embeddings_df.head(3)

(586344, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,20.029926,15.363921,15.476802,17.108240,17.574183,13.780319,0.0,15.937483,9.234217,10.965113,11.433967,25.179270,18.860672,14.596205,14.828723,19.487501
1,14.093033,30.490158,8.972153,12.749017,5.424798,21.921515,0.0,27.035822,10.501605,15.173412,17.048082,19.711557,14.673057,13.568175,9.303555,16.739954
2,11.081604,24.812193,17.010807,17.365387,6.124304,26.997290,0.0,17.478786,16.345547,17.838503,17.856956,4.590561,7.727722,3.016957,14.278222,8.702146


In [27]:

tracks.index[tracks.name == song.lower()] 

Int64Index([], dtype='int64')

In [32]:

def find_neighbors(song):
    '''
    Find the nearest neighbors of a song
    1. Checks for song
    2. Loads and process the embeddings into an array
    3. Trains a nearest neighbors model
    4. Finds the 10 nearest neighbors of the given song
    ARGUMENTS: song in string form
    RETURNS: index of test song, list of prediction indices
    '''
    # 1. Check if song exists: if yes, use first result
    try:
        songs = tracks.index[tracks.name.str.lower() == song.lower()] 
        song_index = songs[0]
    except Exception as e:
        print('ERROR: Not a valid song name')
        

    # 2. Load data ---> ALREADY DONE
    # file = 'embeddings_df_001.csv' # USE CORRECT PATH
    # embeddings = pd.read_csv(file)
    # # Drop extra index column
    # embeddings.drop('Unnamed: 0', axis=1, inplace=True)
    # # Convert dataframe to numpy array
    # encoded_songs = embeddings.to_numpy()

    # 3. Train nearest neighbors model on encodings
    nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
    nn.fit(encoded_songs)

    # 4. Get neigbors of song
    test_encoding = encoded_songs[song_index].reshape(1,-1)
    _, n_indices = nn.kneighbors(test_encoding)
    # First result is often the song, so leave out
    n_indices = n_indices.tolist()[0][1:] 

    return song_index, n_indices

In [36]:
# Song to search by
song = 'yellow submarine'

# Find neighbors
test_song, neighbors = find_neighbors(song)

# Display test song and nearest neighbors
# First display song from search query
ind = test_song
name = tracks.iloc[ind]['name']
artist = tracks.iloc[ind]['artists']
print(f'Test song: {name} by {artist}')
print(f'https://open.spotify.com/track/{tracks.iloc[ind].id}')
# Next display predicitons
print('\nPredictions:')
for i in range(10):
    ind = neighbors[i]
    name = tracks.iloc[ind]['name']
    artist = tracks.iloc[ind]['artists']
    print(f'{name} by {artist}')
    print(f'   https://open.spotify.com/track/{tracks.iloc[ind].id}')

Test song: Yellow Submarine by ['The Beatles']
https://open.spotify.com/track/1tdltVUBkiBCW1C3yB4zyD

Predictions:
Yellow Submarine by ['The Beatles']
   https://open.spotify.com/track/1tdltVUBkiBCW1C3yB4zyD
Will You Still Love Me Tomorrow? - 2011 by ['Amy Winehouse']
   https://open.spotify.com/track/0LbK1bodGLc9xmV1cS9jvL
Guilty Conscience by ['Eminem', 'Dr. Dre']
   https://open.spotify.com/track/6da04YB906XonC6IQNC44u
King of the Dance - 2012 Remaster by ['Robin Trower']
   https://open.spotify.com/track/0pFDHSmprdpCOGS5Nprktr
Green Light by ['Roll Deep']
   https://open.spotify.com/track/1EGZknYcA7uKwMVSWJZpcZ
Monks by ['Frank Ocean']
   https://open.spotify.com/track/3YXVl3FvIrvhgSW9ME3qNf
Dear No One by ['Tori Kelly']
   https://open.spotify.com/track/2wWboDFZJPWXVNlHUbdSVQ
Oops!...I Did It Again by ['Britney Spears']
   https://open.spotify.com/track/6naxalmIoLFWR0siv8dnQQ
Them by ['Misfits']
   https://open.spotify.com/track/5NNG7CVbNyTncH960DMyiu
Phantom of the Opera - Live a